<a href="https://colab.research.google.com/github/mohansameer1983/ComputerVisionProjects/blob/main/Trash_Classification_CV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import cv2
from google.colab.patches import cv2_imshow      # To show images using cv2 module

import pathlib

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Dense, Input, Dropout
from keras.preprocessing.image import ImageDataGenerator

import tensorflow_hub as hub

print("TF version:", tf.__version__)
print("Hub version:", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

TF version: 2.8.0
Hub version: 0.12.0
GPU is NOT AVAILABLE


# Model Selection

In [2]:
  # Select base model  for transfer learning
  model_name = "mobilenet_v3_small_100_224" # @param [ 'resnet_v1_50', 'resnet_v1_101', 'resnet_v1_152', 'resnet_v2_50', 'resnet_v2_101', 'resnet_v2_152',  'mobilenet_v2_100_224', 'mobilenet_v2_130_224', 'mobilenet_v2_140_224', 'mobilenet_v3_small_100_224', 'mobilenet_v3_small_075_224', 'mobilenet_v3_large_100_224', 'mobilenet_v3_large_075_224']

  model_handle_map = {
    "resnet_v1_50": "https://tfhub.dev/google/imagenet/resnet_v1_50/feature-vector/4",
    "resnet_v1_101": "https://tfhub.dev/google/imagenet/resnet_v1_101/feature-vector/4",
    "resnet_v1_152": "https://tfhub.dev/google/imagenet/resnet_v1_152/feature-vector/4",
    "resnet_v2_50": "https://tfhub.dev/google/imagenet/resnet_v2_50/feature-vector/4",
    "resnet_v2_101": "https://tfhub.dev/google/imagenet/resnet_v2_101/feature-vector/4",
    "resnet_v2_152": "https://tfhub.dev/google/imagenet/resnet_v2_152/feature-vector/4",
    "nasnet_large": "https://tfhub.dev/google/imagenet/nasnet_large/feature_vector/4",
    "nasnet_mobile": "https://tfhub.dev/google/imagenet/nasnet_mobile/feature_vector/4",
    "pnasnet_large": "https://tfhub.dev/google/imagenet/pnasnet_large/feature_vector/4",
    "mobilenet_v2_100_224": "https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/4",
    "mobilenet_v2_130_224": "https://tfhub.dev/google/imagenet/mobilenet_v2_130_224/feature_vector/4",
    "mobilenet_v2_140_224": "https://tfhub.dev/google/imagenet/mobilenet_v2_140_224/feature_vector/4",
    "mobilenet_v3_small_100_224": "https://tfhub.dev/google/imagenet/mobilenet_v3_small_100_224/feature_vector/5",
    "mobilenet_v3_small_075_224": "https://tfhub.dev/google/imagenet/mobilenet_v3_small_075_224/feature_vector/5",
    "mobilenet_v3_large_100_224": "https://tfhub.dev/google/imagenet/mobilenet_v3_large_100_224/feature_vector/5",
    "mobilenet_v3_large_075_224": "https://tfhub.dev/google/imagenet/mobilenet_v3_large_075_224/feature_vector/5",
  }

  PIXELS = 224#@param {type:"integer"}
  BATCH_SIZE = 16#@param {type:"integer"}

  model_handle = model_handle_map.get(model_name)

  print(f"Selected model: {model_name} : {model_handle}")

  IMAGE_SIZE = (PIXELS, PIXELS)
  print(f"Input size {IMAGE_SIZE}")


Selected model: mobilenet_v3_small_100_224 : https://tfhub.dev/google/imagenet/mobilenet_v3_small_100_224/feature_vector/5
Input size (224, 224)


# Data Preparation

In [3]:
# Mount Google drive so dataset can be accessed
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [23]:
def load_data(save_dir="./"):
  data_dir = pathlib.Path('/content/drive/MyDrive/Colab Notebooks/data/trash') 

  datagen_kwargs = dict(rescale=1./255, validation_split=.020)
  dataflow_kwargs = dict(target_size=(PIXELS, PIXELS), 
                        batch_size=BATCH_SIZE,
                        interpolation="nearest")

  valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
      **datagen_kwargs)

  valid_generator = valid_datagen.flow_from_directory(
      data_dir, subset="validation", 
      shuffle=False, **dataflow_kwargs)
  
  train_datagen = ImageDataGenerator(
            horizontal_flip=True,
            **datagen_kwargs)

  train_generator = train_datagen.flow_from_directory(
          data_dir,
          class_mode='categorical',
          **dataflow_kwargs)
  return valid_generator, train_generator

VAL_generator, TRAIN_generator = load_data()

Found 4 images belonging to 7 classes.
Found 310 images belonging to 7 classes.


In [24]:
class_names = TRAIN_generator.class_indices.items()
print(class_names)

dict_items([('Aluminium foil', 0), ('Battery', 1), ('Blister pack', 2), ('Bottle', 3), ('Bottle cap', 4), ('Broken glass', 5), ('Can', 6)])


In [9]:
def clean_data(df):

  
  return df

print(train)
train = clean_data(train)
val = clean_data(val)
test = clean_data(test)

print(train.head())
print(test.head())

NameError: ignored

# Modelling

## Build and Train Model


In [25]:
def build_model(trainable=False):
   print("Building model with", model_handle)
   model = keras.Sequential([
        keras.layers.InputLayer(input_shape=IMAGE_SIZE + (3,)),
        hub.KerasLayer(model_handle, trainable=trainable),
        keras.layers.Dropout(rate=0.2),
        keras.layers.Dense(TRAIN_generator.num_classes,
                              kernel_regularizer=tf.keras.regularizers.l2(0.0001))])
   model.build((None,)+IMAGE_SIZE+(3,))
   model.summary()
   return model


model = build_model()

Building model with https://tfhub.dev/google/imagenet/mobilenet_v3_small_100_224/feature_vector/5
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_2 (KerasLayer)  (None, 1024)              1529968   
                                                                 
 dropout_2 (Dropout)         (None, 1024)              0         
                                                                 
 dense_2 (Dense)             (None, 7)                 7175      
                                                                 
Total params: 1,537,143
Trainable params: 7,175
Non-trainable params: 1,529,968
_________________________________________________________________


In [26]:
def compile_model(model):
  model.compile(
    optimizer=tf.keras.optimizers.SGD(lr=0.005, momentum=0.9), 
  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=0.1),
  metrics=['accuracy'])
  return model

model = compile_model(model)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [ ]:
import tensorflow as tf
def train_model(model, num_epochs):

    steps_per_epoch = TRAIN_generator.samples // TRAIN_generator.batch_size
    validation_steps = VAL_generator.samples // VAL_generator.batch_size
    history = model.fit(
      TRAIN_generator,
      epochs=num_epochs, steps_per_epoch=steps_per_epoch,
      validation_data = VAL_generator,
      validation_steps = validation_steps).history
    
    return model, history

model, history = train_model(model, num_epochs=5)

Epoch 1/5
19/19 [==============================] - 86s 4s/step - loss: 1.9383 - accuracy: 0.3231
Epoch 2/5
19/19 [==============================] - 43s 2s/step - loss: 1.3631 - accuracy: 0.5612
